## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn import linear_model, ensemble, model_selection, metrics, tree, neighbors
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn import decomposition, feature_selection, svm, neighbors, datasets, preprocessing, neural_network 
from matplotlib import pyplot

In [ ]:
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from collections import Counter

## Load Data

####  Y matrix (Gene Signatures)

In [ ]:
Y = pd.read_table('Input/consensus-perts.tsv')
Y.set_index('pert_id', inplace=True)
print( Y.shape)

#### Multiple X matricies (fingerprints)

In [ ]:
X1 = pd.read_table('RDKit_fps/Morg1_BMatrix_map_2018_08.tsv')
X1[X1.columns[0]] = X1[X1.columns[0]].astype(str)
X1 = X1.set_index(X1.columns[0])

X2 = pd.read_table('RDKit_fps/TopologicalTorsion_BMatrix_map_2018_08.tsv')
X2[X2.columns[0]] = X2[X2.columns[0]].astype(str)
X2 = X2.set_index(X2.columns[0])

X3 = pd.read_table('RDKit_fps/AtomPair_BMatrix_map_2018_08.tsv')
X3[X3.columns[0]] = X3[X3.columns[0]].astype(str)
X3 = X3.set_index(X3.columns[0])

X4 = pd.read_table('RDKit_fps/MACCs_BMatrix_map_2018_08.tsv')
X4[X4.columns[0]] = X4[X4.columns[0]].astype(str)
X4 = X4.set_index(X4.columns[0])

X5 = pd.read_table('RDKit_fps/RDKfps2_BMatrix_map_2018_08.tsv')
X5[X5.columns[0]] = X5[X5.columns[0]].astype(str)
X5 = X5.set_index(X5.columns[0])

X6 = pd.read_table('RDKit_fps/Avalon_BMatrix_map_2018_08.tsv')
X6[X6.columns[0]] = X6[X6.columns[0]].astype(str)
X6 = X6.set_index(X6.columns[0])

# X7 = pd.read_table('Output/L1000_Scaffolds_2018_07.tsv')
# X7[X7.columns[0]] = X7[X7.columns[0]].astype(str)
# X7 = X7.set_index(X7.columns[0])
# X7 = X7.T

X = pd.concat([X2, X5, X6, X1, X3, X4], axis = 1, sort=True)
X = X.dropna()
X.head()

## Change Y to only have  positive gene signatures

In [ ]:
Y = Y.iloc[:,keep_genes]

## Only get drugs shared between X and Y

In [ ]:
shared_drugs = sorted(list(set(X.index) & set(Y.index)))
X = X.loc[shared_drugs]
Y = Y.loc[shared_drugs]
X = X.values
Y = Y.values

In [ ]:
## Run for regressors without a multitask regressor option
shared_drugs = sorted(list(set(X.index) & set(Y.index)))
X = X.loc[shared_drugs]
Y = Y.loc[shared_drugs]
X = X.values

## Eliminate columns from X based on how many zeros it has

In [ ]:
sns.distplot(list(X.sum(axis=0)))

In [ ]:
x_col_drop = []
x_amt_nonzero = X.sum(axis=0)
for amt in x_amt_nonzero.iteritems():
    if abs(amt[1]) < 500:
        x_col_drop.append(amt[0])
        
X = X.drop(columns = x_col_drop)
X.shape

## Normalize Y

In [ ]:
Y = preprocessing.scale(Y)

## Dimentionality Reduction

In [ ]:
# dr_model = decomposition.LatentDirichletAllocation(n_components=100, learning_method= 'online')
dr_model = decomposition.NMF(n_components=100, init = 'nndsvda')

X_dr = dr_model.fit_transform(X)

In [ ]:
X_dr_df = pd.DataFrame(X_dr)
X_dr_df.head()

In [ ]:
X = X_dr

In [ ]:
X.shape, Y.shape

## Pick the regressor

In [ ]:
# regressor = linear_model.MultiTaskLasso()
# regressor = linear_model.MultiTaskElasticNet()
regressor = linear_model.Ridge(fit_intercept=True, normalize=False, copy_X=True, solver='auto')
# regressor = ensemble.RandomForestRegressor(n_estimators = 60, n_jobs = 7)
# regressor = linear_model.BayesianRidge()
# regressor = linear_model.LassoLars() 
# regressor = ensemble.GradientBoostingRegressor(n_estimators = 5, max_depth= 3, min_samples_split= 3, learning_rate= 0.01, loss= 'ls')
# regressor = svm.SVR(degree = 1, epsilon=.01, kernel = 'poly')
# regressor = ensemble.AdaBoostRegressor(base_estimator=None, n_estimators=50, learning_rate=1.0, loss='linear', random_state=None)
# regressor = neural_network.MLPRegressor(hidden_layer_sizes=(100, ))
# regressor = neighbors.KNeighborsRegressor(n_neighbors=100)


## Run the model

### Normal run model

In [ ]:
r2ss = []
cv = model_selection.KFold(n_splits=4, shuffle=True)
start = time.time()
for train_idx, test_idx in cv.split(X):
    X_train, Y_train = X[train_idx], Y[train_idx]
    
    X_test, Y_test = X[test_idx], Y[test_idx]
    
    regressor.fit(X_train, Y_train)
    Y_test_pred = regressor.predict(X_test)
    
    r2s = metrics.r2_score(Y_test, Y_test_pred, multioutput='raw_values')
    r2ss.append(r2s)
    
    end = time.time()
    print(end-start)

### Run Model with Feature Selection 

In [ ]:
r2ss = []
cv = model_selection.KFold(n_splits=4, shuffle=True)
start = time.time()
for train_idx, test_idx in cv.split(X):
    X_train, Y_train = X[train_idx], Y[train_idx]

    # k-nearest neighbor classifier to evaluate  predictabilities of individual features in X using Y
    knn = neighbors.KNeighborsClassifier(n_neighbors=10)
    knn.fit(Y_train, X_train)
    # Use F1-score to evaluate the predictability of x
    x_fs_scores = metrics.f1_score(knn.predict(Y_train), X_train, average=None)
    # Drop the bottom 20% least predictable features
    mask_features_to_keep = x_fs_scores > np.percentile(x_fs_scores, 40)
    X_train = X_train[:, mask_features_to_keep]
    
    # Drop those features from X_test
    X_test, Y_test = X[test_idx][:, mask_features_to_keep], Y[test_idx]
    
    # Fit the multi-task regression model 
    regressor.fit(X_train, Y_train)
    Y_test_pred = regressor.predict(X_test)
    
    r2s = metrics.r2_score(Y_test, Y_test_pred, multioutput='raw_values')
    r2ss.append(r2s)
    
    end = time.time()
    print(end-start)

In [ ]:
(Y_test.shape, Y_test_pred.shape)

## Plot the Average R^2

In [ ]:
data = np.array(r2ss)
aver_data = np.average(data, axis=0)
sns.distplot(aver_data)

In [ ]:
avg = np.average(aver_data)
avg

## Analyze score and R^2

In [ ]:
regressor.score(X_test, Y_test)

In [ ]:
## Feature selection only
sns.distplot(x_fs_scores)

## Plot the estimates

In [ ]:
plt.figure()
s = 35
a = .7

Y_train_pred = regressor.predict(X_train)

plt.scatter(Y_test[:, 0], Y_test[:, 1], edgecolor='k',
            c="blue", s=s, marker="s", alpha=a, label="Data")
plt.scatter(Y_test_pred[:, 0], Y_test_pred[:, 1], edgecolor='k',
            c="red", s=s, marker="^", alpha=a,
            label="RF score=%.2f" % regressor.score(X_test, Y_test))
plt.scatter(Y_train_pred[:, 0], Y_train_pred[:, 1], edgecolor='k',
            c="yellow", s=s, marker="o", alpha=a,
            label="RF score=%.2f" % regressor.score(X_train, Y_train))

plt.xlim([-4, 4])
plt.ylim([-4, 4])
plt.title("Visualizing Random Forest")
plt.legend()

## For Regressors without Multitask

In [ ]:
r2ss = []
cv = model_selection.KFold(n_splits=4, shuffle=True)
start = time.time()

for i in list(Y.columns):
    Y_i = Y[i]
    for train_idx, test_idx in cv.split(X):
        X_train, Y_train = X[train_idx], Y_i[train_idx]
        X_test, Y_test = X[test_idx], Y_i[test_idx]

        regressor.fit(X_train, Y_train)
        Y_test_pred = regressor.predict(X_test)

        r2s = metrics.r2_score(Y_test, Y_test_pred, multioutput='raw_values')
        r2ss.append(r2s)

end = time.time()
print(end-start)

new_r2ss = [r2ss[i][0] for i in range(len(r2ss))]

In [ ]:
sns.distplot(new_r2ss)

In [ ]:
np.average(new_r2ss)

## List of genes giving postive R^2

In [ ]:
desired_genes = []
for array in r2ss:
    for gene in array:
        if gene >0.1:
            desired_genes.append(list(array).index(gene))

In [ ]:
len(set(desired_genes))

In [ ]:
print(sorted(desired_genes)[:10])

In [ ]:
cnt = Counter()
for gene in desired_genes:
    cnt[gene] += 1
dict(cnt)
keep_genes = []
for entry in cnt.items():
    if entry[1] >= 3:
        keep_genes.append(entry[0])

In [ ]:
len(keep_genes)

## Plot estimator vs time/r^2 for random forest

In [ ]:
estimator = [5,10,20,50,75,100,400]
time = [8.63,17.88,31.06,73.46,110.39,143.9,641.10]
r2 = [-0.12,-0.038,0.0105,0.055,0.051,0.0307,0.0225]

fig, ax1 = plt.subplots()

plt.xlabel('Number of estimators', fontsize= 14)
plt.xticks(fontsize = 11)

ax1.plot(estimator,time,color='purple', linewidth = 2.5)
ax1.tick_params(axis = 'y', labelcolor = 'purple', labelsize = 11.0)
plt.ylabel('Time (s)', fontsize = 14, color = 'purple')

ax2 = ax1.twinx()

ax2.plot(estimator, r2, color = 'blue', linewidth = 2.0)
ax2.tick_params(axis = 'y', labelcolor = 'blue', labelsize = 11.0)
plt.ylabel('R^2 Value', fontsize = 14, color = 'blue')

fig.tight_layout()
plt.title('Estimators vs. Time and R^2 Value', fontsize = 15)
plt.savefig('Presentation_figures/estimators_time_r2_ranfor' ,bbox_inches = 'tight')
plt.show()
